In [327]:
import numpy as np
import pandas as pd

In [328]:
df = pd.read_csv('Gurgaon_properties_feature_selection.csv')

In [329]:
df.head()

property_type  sector  bedRoom  bathroom  balcony  agePossession  \
0            0.0   111.0      2.0       2.0      3.0            3.0   
1            0.0   103.0      3.0       3.0      3.0            3.0   
2            0.0     9.0      2.0       2.0      4.0            3.0   
3            0.0     4.0      4.0       5.0      4.0            3.0   
4            0.0    76.0      2.0       2.0      2.0            1.0   

   built_up_area  servant room  pooja room  study room  store room  \
0         1148.0           0.0         1.0         0.0         0.0   
1         1540.0           0.0         0.0         1.0         0.0   
2         1201.0           0.0         1.0         0.0         0.0   
3         2600.0           1.0         1.0         0.0         0.0   
4          633.0           0.0         0.0         0.0         0.0   

   furnishing_type  luxury_category  floor_category  price  
0              0.0              2.0             2.0   0.73  
1              0.0              0.0             2.0   1.29  
2              0.0              0.0             0.0   0.75  
3              1.0              0.0             2.0   2.50  
4              0.0              1.0             0.0   0.60

In [330]:
df['sector'] = df['sector'].astype('int').astype('object')

In [331]:
df['sector'].value_counts()

111    152
97     108
4      107
105    100
78      93
      ... 
100      3
84       3
32       2
23       1
42       1
Name: sector, Length: 113, dtype: int64

In [295]:
df = df[~(df['sector'] == 42) & ~(df['sector'] == 23) & ~(df['sector'] == 32)]

In [296]:
df['sector'].value_counts()

111    152
97     108
4      107
105    100
78      93
      ... 
48       5
37       5
100      3
24       3
84       3
Name: sector, Length: 110, dtype: int64

In [318]:
df.shape

(3550, 15)

In [297]:
X = df.drop(columns='price')
y = df['price']

In [298]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR

In [299]:
col_ohe_encode = ['sector','balcony','agePossession','furnishing_type','luxury_category','floor_category']

In [300]:
y_transformed = np.log1p(y)

In [301]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room','pooja room','study room']),
        ('cat', OneHotEncoder(drop='first'), col_ohe_encode)
    ], 
    remainder='passthrough'
)

In [303]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [304]:
kfold = KFold(n_splits=10, shuffle=True, random_state=53)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2',error_score='raise')

In [305]:
scores.mean()

0.8787922518814912

In [306]:
scores

array([0.90316176, 0.88186946, 0.85699643, 0.90111315, 0.85366643,
       0.87555802, 0.86293015, 0.88261106, 0.89534514, 0.87467092])

In [307]:
from sklearn.model_selection import train_test_split

In [308]:
X_train,X_test,y_train,y_test = train_test_split(X,y_transformed,random_state=42)

In [309]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room', 'store room',
                                                   'pooja room',
                                                   'study room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                            

In [313]:
y_pred = pipeline.predict(X_test)

In [316]:
y_pred = np.expm1(y_pred)

In [317]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5844374352857585